# Ingesting PDF

In [2]:
%pip install --q unstructured langchain langchain-community
%pip install --q "unstructured[all-docs]" ipywidgets tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.1.1
[notice] To update, run: d:\Data_sci_project\.venv\Scripts\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.1.1
[notice] To update, run: d:\Data_sci_project\.venv\Scripts\python.exe -m pip install --upgrade pip


In [7]:
from langchain_community.document_loaders import UnstructuredPDFLoader

In [8]:
path = "data\EazyCar_Product_knowledge_for_AI_chatbot.pdf"
loader = UnstructuredPDFLoader(path)
docs = loader.load()
# print(docs[0].page_content)

# Vecter Embedding

In [9]:
# !ollama pull nomic-embed-text:v1.5
!ollama list
# !pip install -q chromadb
# !pip install -q langchain-text-splitters
# !ollama pull deepseek-r1:1.5b

NAME                     ID              SIZE      MODIFIED       
deepseek-r1:1.5b         e0979632db5a    1.1 GB    14 minutes ago    
nomic-embed-text:v1.5    0a109f422b47    274 MB    2 days ago        


In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
chucks = text_splitter.split_documents(docs)

In [11]:
#Add to vector database
vector_db = Chroma.from_documents(
    documents=chucks,
    embedding=OllamaEmbeddings(model="nomic-embed-text:v1.5",show_progress=True),
    collection_name="Eazycar_db"
)

C:\Users\AVI03\AppData\Local\Temp\ipykernel_18084\2536676596.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model="nomic-embed-text:v1.5",show_progress=True),
OllamaEmbeddings: 100%|██████████| 7/7 [00:18<00:00,  2.63s/it]


# Retrieval

In [12]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [13]:
local_model = "deepseek-r1:1.5b"
llm = ChatOllama(model=local_model)
QUERRY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}
    """
)

C:\Users\AVI03\AppData\Local\Temp\ipykernel_18084\1972936892.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=local_model)


In [14]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(),
    llm,
    prompt=QUERRY_PROMPT
)

#RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
""" 
prompt = ChatPromptTemplate.from_template(template)

In [16]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [17]:
chain.invoke(input(""))

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.13s/it]


"<think>\nFirst, I need to determine whether Eazy Car is a subscription service that offers cars or if it's about taxes. The context provided includes information on what Eazy Car is and how it handles insurance, taxes, and vehicle ownership. It mentions that Eazy Car provides flexible terms with a fixed monthly rate for all types of vehicles, including private and corporate use. It also covers how to apply for the service, the required documents, payment methods, security deposits, tax deductions, and cancellation procedures. The information is structured in several parts, each focusing on different aspects like service types, pricing, application, documentation, insurance, taxes, maintenance, and customer inquiries.\n\nSince Eazy Car focuses on offering cars as part of a subscription service with various terms and conditions related to payment, security, taxes, and service specifics, the primary focus is on its car offerings and management rather than general tax or insurance informa